## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
import os
current_directory = os.getcwd()
print(current_directory) 

C:\Users\clava\World_Weather_Analysis\Vacation_Search


In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"C:\Users\clava\World_Weather_Analysis\Weather_Database\WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Date
0,0,Tahta,26.7693,31.5021,68.40,46,0,9.31,clear sky,EG,2021-10-24 22:57:14
1,1,Albany,42.6001,-73.9662,52.81,75,95,2.75,overcast clouds,US,2021-10-24 22:54:00
2,2,Yantongshan,43.2919,126.0094,40.10,76,37,5.86,scattered clouds,CN,2021-10-24 22:57:15
3,3,Nikolskoye,59.7035,30.7861,34.88,78,88,7.09,overcast clouds,RU,2021-10-24 22:54:40
4,4,Hermanus,-34.4187,19.2345,55.90,76,0,9.51,clear sky,ZA,2021-10-24 22:51:03


In [14]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [15]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Date
6,6,Puerto Ayora,-0.7393,-90.3518,78.76,87,100,10.33,overcast clouds,EC,2021-10-24 22:54:27
7,7,Isangel,-19.5500,169.2667,80.71,69,75,5.75,broken clouds,VU,2021-10-24 22:51:21
8,8,Hilo,19.7297,-155.0900,84.27,83,32,3.00,scattered clouds,US,2021-10-24 22:54:05
10,10,Tecoanapa,16.5167,-98.7500,80.65,82,89,6.44,overcast clouds,MX,2021-10-24 22:57:16
17,17,Kapaa,22.0752,-159.3190,84.18,77,53,5.01,broken clouds,US,2021-10-24 22:54:32
26,26,Cayenne,4.9333,-52.3333,78.84,89,90,10.00,moderate rain,GF,2021-10-24 22:54:22
33,33,Mtwara,-10.2667,40.1833,76.42,89,23,7.52,light rain,TZ,2021-10-24 22:57:19
37,37,Guanambi,-14.2233,-42.7814,78.35,70,99,6.06,overcast clouds,BR,2021-10-24 22:57:20
39,39,Nioro,13.3500,-15.7500,79.81,81,100,6.17,overcast clouds,GM,2021-10-24 22:53:53
42,42,Mount Isa,-20.7333,139.5000,80.37,50,100,13.80,light rain,AU,2021-10-24 22:50:20


In [16]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             171
City                   171
Lat                    171
Lng                    171
Max Temp               171
Humidity               171
Cloudiness             171
Wind Speed             171
Current Description    171
Country                170
Date                   171
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.reset_index(drop=True)
clean_df.count()

Unnamed: 0             170
City                   170
Lat                    170
Lng                    170
Max Temp               170
Humidity               170
Cloudiness             170
Wind Speed             170
Current Description    170
Country                170
Date                   170
dtype: int64

In [18]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Puerto Ayora,EC,78.76,overcast clouds,-0.7393,-90.3518,
7,Isangel,VU,80.71,broken clouds,-19.5500,169.2667,
8,Hilo,US,84.27,scattered clouds,19.7297,-155.0900,
10,Tecoanapa,MX,80.65,overcast clouds,16.5167,-98.7500,
17,Kapaa,US,84.18,broken clouds,22.0752,-159.3190,
26,Cayenne,GF,78.84,moderate rain,4.9333,-52.3333,
33,Mtwara,TZ,76.42,light rain,-10.2667,40.1833,
37,Guanambi,BR,78.35,overcast clouds,-14.2233,-42.7814,
39,Nioro,GM,79.81,overcast clouds,13.3500,-15.7500,
42,Mount Isa,AU,80.37,light rain,-20.7333,139.5000,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
                
        print("Hotel not found... skipping.")
        

        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Puerto Ayora,EC,78.76,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
7,Isangel,VU,80.71,broken clouds,-19.5500,169.2667,Tanna Lodge
8,Hilo,US,84.27,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
10,Tecoanapa,MX,80.65,overcast clouds,16.5167,-98.7500,Hotel Manglares
17,Kapaa,US,84.18,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [21]:
# 8a. Create the output File (CSV)
output_data_file = ("WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]
locations

,Lat,Lng
6,-0.7393,-90.3518
7,-19.5500,169.2667
8,19.7297,-155.0900
10,16.5167,-98.7500
17,22.0752,-159.3190
...,...,...
675,-21.3500,165.1500
677,16.3333,-94.0667
678,5.1053,-1.2466
686,8.5711,81.2335


In [23]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))